In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
tf.set_random_seed(55)
np.random.seed(55)

In [ ]:
class FNN(object):
    """Build a general FeedForward neural network
    Parameters
    ----------
    learning_rate: float
    drop_out: float
    Layers: list
    N_hidden: list
    D_input: int
    D_label: int
    Task_type: string
        'regression' or 'classification'
    L2_lambda: float
    """
    def __init__(self, learning_rate, drop_out, Layers, N_hidden,
                 D_input, D_label, Task_type='regression', L2_lambda=0.0):
        #var
        self.learning_rate = learning_rate
        self.drop_out = drop_out
        self.Layers = Layers
        self.N_hidden = N_hidden
        self.D_input = D_input
        self.D_label = D_label
        #类型控制loss函数的选择
        self.Task_type = Task_type
        #L2 regularization的惩罚强弱，过高会使输出倾向于0
        self.L2_lambda = L2_lambda
        #用于存放锁累积的每层L2 regularization
        self.l2_penalty = tf.constant(0.0)
        
        #用于生成缩放图，括号里起名字
        with tf.name_scope('Input'):
            self.inputs = tf.placeholder(tf.float32,[None,D_input],name='inputs')
        with tf.name_scope('Label'):
            self.labels = tf.placeholder(tf.float32,[None,D_label],name='labels')
        with tf.name_scope('keep_rate'):
            self.drop_keep_rate = tf.placeholder(tf.float32,name='dropout_keep')
        
        #初始化时直接生成，build方法是后面会建立的
        self.build('F')
        
    def weight_init(self,shape):
        #shape: list [in_dim, out_dim]
        #can change initialization here
        initial = tf.random_uniform(shape,minval=-np.sqrt(5)*np.sqrt(1.0/shape[0]),maxval=np.sqrt(5)*np.sqrt(1.0/shape[0]))
        return tr.Variable(initial)
    
    def bias_init(self, shape):
        #can change initialization here
        initial = tf.constant(0.1,shape=shape)
        return tf.Variable(initial)
    
    def varialbe_summaries(self, var, name):
        with tf.name_scope(name+'_summaries'):
            mean = tf.reduce_mean(var)
            tf.scalar_summary('mean/'+name, mean)
        with tf.name_scope(name+'_stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        #record the change data after each training
        tf.scalar_summary('_stddev/'+name,stddev)
        tf.scalar_summary('_max/'+name,tf.reduce_max(var))
        tf.scalar_summary('_min/'+name,tf.reduce_min(var))
        tf.histogram_summary(name,var)
    
    def layer(self, in_tensor, in_dim, out_dim, layer_name, act=tf.nn.relu):
        with tf.name_scope(layer_name):
            with tf.name_scope(layer_name+'_weights'):
                # initial by function weight_init()
                weights = self.weight_init([in_dim, out_dim])
                # 存放每一个权重W
                self.W.append(weights)
                # 对权重进行统计
                self.variable_summaries(weights, layer_name+'/weights')
            with tf.name_scope(layer_name+'_biases'):
                biases = self.bias_init([out_dim])
                self.variable_summaries(biases, layer_name+'/biases')
            with tf.name_scope(layer_name+'_Wx_plus_b'):
                #calculate Wx+b
                pre_activate = tf.matmul(in_tensor, weights) + biases
                #记录直方图
                tf.histogram_summary(layer_name+'/pre_activations',pre_activate)
                
            #计算a(Wx+b)
            activations = act(pre_activate, name='activation')
            tf.histogram_summary(layer_name+'/activations',activations)
        #返回该层的输出以及权重W的l2
        return activations, tf.nn.l2_loss(weights)
    
